In [2]:
%run ../../connection.python.py

In [3]:
path_adobe =  '/user/matteo.ballerinipuviani/dig_gen_giu_2018_df_pulito.parquet'
#path_test_3 =  "/user/emma.cambieri/GOLDEN_SET_DIGITALIZZAZIONE_2019_FINALI/golden_lugl_dic_1_0scelti_e_Null.parquet"


path_model_generico = "/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE/model_generici_def.parquet"
path_model_fan = "/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE/model_fan_def.parquet"
path_model_social = "/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE/model_social_definitivo_davvero.parquet"



path_spreading_model_generico = "/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE_LUG_DIC/probability_generico_con_decili.parquet"
path_spreading_model_fan = "/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE_LUG_DIC/probability_fan_con_decili.parquet"
path_spreading_model_social = "/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE_LUG_DIC/probability_social_con_decili.parquet"



In [4]:
adobe = spark.read.format('parquet').load(path_adobe).persist()

#testData3 = spark.read.format('parquet').load(path_test_3).persist()

###############

In [5]:
adobe = adobe.withColumnRenamed('social_network', 'social_network_da_scartare')
adobe.createOrReplaceTempView('adobe')
adobe= spark.sql('SELECT *, CASE WHEN(social_network_da_scartare is null) THEN "no_social" ELSE social_network_da_scartare END AS social_network FROM adobe')
adobe= adobe.drop('social_network_da_scartare')

In [5]:
#prova_social = spark.sql('SELECT social_network_2, count(ext_id) as count_ext_id FROM testData group by social_network_2')

In [6]:
#prova_social.show()

###############

In [6]:
string_columns= ["social_network", "tipo_device", "ricorrenza_corporate", "ricorrenza_tg24", "ricorrenza_sport",
                      "ricorrenza_appgtv", "ricorrenza_appxfactor", "ricorrenza_appsport", "ricorrenza_appwsc",
                      "ricorrenza_corporate_appwsc_appgtv", "ricorrenza_editoriale", "ricorrenza_web", "ricorrenza_app",
                      "ricorrenza_totale"]
for c in string_columns:
    adobe = adobe.withColumn(c, adobe[c].cast('string'))

avg_columns= ["media_visite_corporate","media_visite_tg24", "media_visite_sport",
                    "media_visite_appgtv","media_visite_appxfactor","media_visite_appsport","media_visite_appwsc",
                    "avg_visite_mese_web", "percentuale_mobile",
                    "percentuale_desktop", "percentuale_other", "durata_media_visite_corporate", "durata_media_visite_tg24",
                     "durata_media_visite_sport", "durata_media_visite_appgtv", "durata_media_visite_appxfactor", "durata_media_visite_appsport",
                    "durata_media_visite_appwsc", "avg_visite_mese_corporate_appwsc_appgtv", "avg_visite_mese_editoriale", "avg_visite_mese_app",
                     "avg_sec_corporate_appwsc_appgtv","avg_sec_editoriale", "avg_sec_web", "avg_sec_app"]

for cols in avg_columns:
    adobe= adobe.withColumn(cols, adobe[cols].cast('float'))
        
integ_columns = adobe.drop("ext_id","social_network", "tipo_device", "ricorrenza_corporate", "ricorrenza_tg24", "ricorrenza_sport",
                      "ricorrenza_appgtv", "ricorrenza_appxfactor", "ricorrenza_appsport", "ricorrenza_appwsc",
                      "ricorrenza_corporate_appwsc_appgtv", "ricorrenza_editoriale", "ricorrenza_web", "ricorrenza_app",
                      "ricorrenza_totale", "media_visite_corporate","media_visite_tg24", "media_visite_sport",
                    "media_visite_appgtv","media_visite_appxfactor","media_visite_appsport","media_visite_appwsc",
                    "avg_visite_mese_web", "percentuale_mobile",
                    "percentuale_desktop", "percentuale_other", "durata_media_visite_corporate", "durata_media_visite_tg24",
                     "durata_media_visite_sport", "durata_media_visite_appgtv", "durata_media_visite_appxfactor", "durata_media_visite_appsport",
                    "durata_media_visite_appwsc", "avg_visite_mese_corporate_appwsc_appgtv", "avg_visite_mese_editoriale", "avg_visite_mese_app",
                     "avg_sec_corporate_appwsc_appgtv", "avg_sec_editoriale", "avg_sec_web", "avg_sec_app").columns


for col_int in integ_columns:
  adobe = adobe.withColumn(col_int,adobe[col_int].cast("integer"))

#adobe.printSchema()

## Creo adobe_test e adobe_0_esclusi dallo spreading del modello:

In [7]:
#0 da escludere dal test:
adobe_0_esclusi= adobe.filter('tipo_device == "desktop" and visite_app == 0 and visite_web == 1')
adobe_0_esclusi = adobe_0_esclusi.withColumn('score_digital', lit(0))
adobe_0_esclusi = adobe_0_esclusi.withColumn('prob_1_int',adobe_0_esclusi['score_digital'].cast("long"))

In [8]:
#adobe_0_esclusi.count()

In [9]:
adobe_test = adobe.filter('tipo_device != "desktop" or visite_app != 0 or visite_web != 1')

In [11]:
#adobe_test.count()

In [12]:
#col_test = testData.columns
#col_golden_train = testData3.columns

#print(len(set(col_test).intersection(col_golden_train)))

#testData2.printSchema()

In [ ]:
#print(len(testData.columns))      

In [ ]:
#testData.printSchema()

In [13]:
adobe_test = adobe_test.drop('totale_visite',  'numero_mesi_corporate', 'numero_mesi_tg24', 'numero_mesi_sport', 'numero_mesi_appwsc',  'numero_mesi_appgtv', 'numero_mesi_appsport', 'numero_mesi_appxfactor', 'visite_totali_corporate',  'visite_totali_tg24', 'visite_totali_sport', 'visite_totali_appwsc', 'visite_totali_appgtv', 'visite_totali_appsport', 'visite_totali_appxfactor',  'cookie_corporate', 'cookie_tg24' , 'cookie_sport' , 'secondi_totali_corporate'
 , 'secondi_totali_tg24' , 'secondi_totali_sport' , 'secondi_totali_appwsc' , 'secondi_totali_appgtv' , 'secondi_totali_appsport' , 'secondi_totali_appxfactor' , 'visite_corporate_appwsc_appgtv'
 , 'visite_editoriale' , 'visite_web' , 'visite_app' , 'max_mesi_corporate_appwsc_appgtv' , 'max_mesi_editoriale' , 'max_mesi_web' , 'max_mesi_app' , 'secondi_corporate_appwsc_appgtv'
 , 'secondi_editoriale' , 'secondi_web' , 'secondi_app', 'score_utilizzo',  'visite_settimana_corporate'
 , 'visite_weekend_corporate'
 , 'visite_settimana_tg24'
 , 'visite_weekend_tg24'
 , 'visite_settimana_sport'
 , 'visite_weekend_sport'
 , 'visite_settimana_appwsc'
 , 'visite_weekend_appwsc'
 , 'visite_settimana_appgtv'
 , 'visite_weekend_appgtv'
 , 'visite_settimana_appsport'
 , 'visite_weekend_appsport'
 , 'visite_settimana_appxfactor'
 , 'visite_weekend_appxfactor'
 , 'visite_corporate_notte'
 , 'visite_corporate_mattina'
 , 'visite_corporate_pomeriggio'
 , 'visite_corporate_sera'
 , 'visite_tg24_notte'
 , 'visite_tg24_mattina'
 , 'visite_tg24_pomeriggio'
 , 'visite_tg24_sera'
 , 'visite_sport_notte'
 , 'visite_sport_mattina'
 , 'visite_sport_pomeriggio'
 , 'visite_sport_sera'
 , 'visite_appwsc_notte'
 , 'visite_appwsc_mattina'
 , 'visite_appwsc_pomeriggio'
 , 'visite_appwsc_sera'
 , 'visite_appgtv_notte'
 , 'visite_appgtv_mattina'
 , 'visite_appgtv_pomeriggio'
 , 'visite_appgtv_sera'
 , 'visite_appsport_notte'
 , 'visite_appsport_mattina'
 , 'visite_appsport_pomeriggio'
 , 'visite_appsport_sera'
 , 'visite_appxfactor_notte'
 , 'visite_appxfactor_mattina'
 , 'visite_appxfactor_pomeriggio'
 , 'visite_appxfactor_sera', 'score_digital')

In [15]:
adobe_0_esclusi = adobe_0_esclusi.drop('totale_visite',  'numero_mesi_corporate', 'numero_mesi_tg24', 'numero_mesi_sport', 'numero_mesi_appwsc',  'numero_mesi_appgtv', 'numero_mesi_appsport', 'numero_mesi_appxfactor', 'visite_totali_corporate',  'visite_totali_tg24', 'visite_totali_sport', 'visite_totali_appwsc', 'visite_totali_appgtv', 'visite_totali_appsport', 'visite_totali_appxfactor',  'cookie_corporate', 'cookie_tg24' , 'cookie_sport' , 'secondi_totali_corporate'
 , 'secondi_totali_tg24' , 'secondi_totali_sport' , 'secondi_totali_appwsc' , 'secondi_totali_appgtv' , 'secondi_totali_appsport' , 'secondi_totali_appxfactor' , 'visite_corporate_appwsc_appgtv'
 , 'visite_editoriale' , 'visite_web' , 'visite_app' , 'max_mesi_corporate_appwsc_appgtv' , 'max_mesi_editoriale' , 'max_mesi_web' , 'max_mesi_app' , 'secondi_corporate_appwsc_appgtv'
 , 'secondi_editoriale' , 'secondi_web' , 'secondi_app', 'score_utilizzo',  'visite_settimana_corporate'
 , 'visite_weekend_corporate'
 , 'visite_settimana_tg24'
 , 'visite_weekend_tg24'
 , 'visite_settimana_sport'
 , 'visite_weekend_sport'
 , 'visite_settimana_appwsc'
 , 'visite_weekend_appwsc'
 , 'visite_settimana_appgtv'
 , 'visite_weekend_appgtv'
 , 'visite_settimana_appsport'
 , 'visite_weekend_appsport'
 , 'visite_settimana_appxfactor'
 , 'visite_weekend_appxfactor'
 , 'visite_corporate_notte'
 , 'visite_corporate_mattina'
 , 'visite_corporate_pomeriggio'
 , 'visite_corporate_sera'
 , 'visite_tg24_notte'
 , 'visite_tg24_mattina'
 , 'visite_tg24_pomeriggio'
 , 'visite_tg24_sera'
 , 'visite_sport_notte'
 , 'visite_sport_mattina'
 , 'visite_sport_pomeriggio'
 , 'visite_sport_sera'
 , 'visite_appwsc_notte'
 , 'visite_appwsc_mattina'
 , 'visite_appwsc_pomeriggio'
 , 'visite_appwsc_sera'
 , 'visite_appgtv_notte'
 , 'visite_appgtv_mattina'
 , 'visite_appgtv_pomeriggio'
 , 'visite_appgtv_sera'
 , 'visite_appsport_notte'
 , 'visite_appsport_mattina'
 , 'visite_appsport_pomeriggio'
 , 'visite_appsport_sera'
 , 'visite_appxfactor_notte'
 , 'visite_appxfactor_mattina'
 , 'visite_appxfactor_pomeriggio'
 , 'visite_appxfactor_sera', 'score_digital')

In [16]:
adobe_test = adobe_test.fillna(0)
adobe_0_esclusi=adobe_0_esclusi.fillna(0)

## Carico i modelli e creo la pipeline per codificare le variabili:

In [17]:
#Import useful packages:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

In [18]:
model_generico = RandomForestClassificationModel.load(path_model_generico)
model_fan = RandomForestClassificationModel.load(path_model_fan)
model_social = RandomForestClassificationModel.load(path_model_social)

In [19]:
steps1 =[]
CategoricalCol_in = ["social_network", "tipo_device", "ricorrenza_corporate", "ricorrenza_tg24", "ricorrenza_sport",
                      "ricorrenza_appgtv", "ricorrenza_appxfactor", "ricorrenza_appsport", "ricorrenza_appwsc",
                      "ricorrenza_corporate_appwsc_appgtv", "ricorrenza_editoriale", "ricorrenza_web", "ricorrenza_app",
                      "ricorrenza_totale"]
CategoricalCol_out = [ "social_network_class_vec", "tipo_device_class_vec", "ricorrenza_corporate_class_vec", "ricorrenza_tg24_class_vec", "ricorrenza_sport_class_vec",
                      "ricorrenza_appgtv_class_vec", "ricorrenza_appxfactor_class_vec", "ricorrenza_appsport_class_vec", "ricorrenza_appwsc_class_vec",
                      "ricorrenza_corporate_appwsc_appgtv_class_vec", "ricorrenza_editoriale_class_vec", "ricorrenza_web_class_vec", "ricorrenza_app_class_vec",
                      "ricorrenza_totale_class_vec"]
 
indexers = [StringIndexer(inputCol=x, outputCol=x+'Index')
            for x in CategoricalCol_in ]
 
encoders = [OneHotEncoder(dropLast=False, inputCol=x+"Index", outputCol=y)
for x,y in zip(CategoricalCol_in, CategoricalCol_out)]
steps1 = [[i,j] for i,j in zip(indexers, encoders)]
steps1 = [i for sublist in steps1 for i in sublist]

numericCols = adobe_test.drop("ext_id", "social_network", "tipo_device", "ricorrenza_corporate", "ricorrenza_tg24", "ricorrenza_sport",
                      "ricorrenza_appgtv", "ricorrenza_appxfactor", "ricorrenza_appsport", "ricorrenza_appwsc",
                      "ricorrenza_corporate_appwsc_appgtv", "ricorrenza_editoriale", "ricorrenza_web", "ricorrenza_app",
                      "ricorrenza_totale").columns

assemblerInputs = [c + "_class_vec" for c in CategoricalCol_in] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
steps1 += [assembler]

#Pipeline:  
pipeline1 = Pipeline().setStages(steps1)

In [20]:
# Applico pipeline ad adobe_test:
pipelineModel_final = pipeline1.fit(adobe_test)
final= pipelineModel_final.transform(adobe_test)

In [21]:
probab_col = ['prob_1_int']
assembler_golden0 = VectorAssembler(inputCols=probab_col, outputCol="prob_1")

In [22]:
zeri_esclusi = assembler_golden0.transform(adobe_0_esclusi)
zeri_esclusi = zeri_esclusi.drop('prob_1_int')

## Modello Generico

In [23]:
predictions_class_generico = model_generico.transform(final)

In [24]:
from pyspark.ml.feature import VectorSlicer

slicer = VectorSlicer(inputCol="probability", outputCol="prob_1", indices=[1])

predictions_class_generico = slicer.transform(predictions_class_generico)

#predictions_class_generico.select("probability", "prob_1").show()

### Unione 0 con predictions modello generico

In [25]:
predictions_class_generico.createOrReplaceTempView('generico')

In [26]:
predictions_class_generico1 = predictions_class_generico.drop('features', 'rawPrediction', 'probability', 'prediction',"social_network_class_vec", "tipo_device_class_vec", "ricorrenza_corporate_class_vec", "ricorrenza_tg24_class_vec", "ricorrenza_sport_class_vec",
                      "ricorrenza_appgtv_class_vec", "ricorrenza_appxfactor_class_vec", "ricorrenza_appsport_class_vec", "ricorrenza_appwsc_class_vec",
                      "ricorrenza_corporate_appwsc_appgtv_class_vec", "ricorrenza_editoriale_class_vec", "ricorrenza_web_class_vec", "ricorrenza_app_class_vec",
                      "ricorrenza_totale_class_vec", "social_networkIndex", "tipo_deviceIndex", "ricorrenza_corporateIndex", "ricorrenza_tg24Index", "ricorrenza_sportIndex",
                      "ricorrenza_appgtvIndex", "ricorrenza_appxfactorIndex", "ricorrenza_appsportIndex", "ricorrenza_appwscIndex",
                      "ricorrenza_corporate_appwsc_appgtvIndex", "ricorrenza_editorialeIndex", "ricorrenza_webIndex", "ricorrenza_appIndex",
                      "ricorrenza_totaleIndex")

In [27]:
predictions_class_generico1.createOrReplaceTempView('predictions_class_generico1')
zeri_esclusi.createOrReplaceTempView('zeri_esclusi')

In [28]:
predictions_finale = spark.sql('select * from predictions_class_generico1 union all select * from zeri_esclusi ')

In [29]:
#print(len(zeri_esclusi.columns))

52


In [24]:
#col_test = predictions_class_generico1.columns
#col_golden_train = zeri_esclusi.columns

#print(len(set(col_test).intersection(col_golden_train)))

In [30]:
predictions_finale.createOrReplaceTempView('predictions_finale')

In [31]:
probab_class_asc = spark.sql('SELECT ext_id, prob_1 FROM predictions_finale ORDER BY prob_1').persist()

In [33]:
#probab_class_asc.count()

In [34]:
probab_class_asc.createOrReplaceTempView('probab_class_asc')
probab_class_asc_indexed = spark.sql('select row_number() over (order by prob_1) as num, * from probab_class_asc').persist()

In [35]:
#probab_class_asc_indexed.show(100)

In [36]:
probab_class_asc_indexed.createOrReplaceTempView('probab_class_asc_indexed')

In [31]:
## ATTENZIONE! Da AUTOMATIZZARE!

probab_class_asc_index_decili = spark.sql('SELECT *, CASE WHEN (num between 0 and 297136) THEN 1 WHEN (num between 297137 and 594272) THEN 2 WHEN (num between 594273 and 891408) THEN 3 WHEN (num between 891409 and 1188544) THEN 4 WHEN (num between 1188545 and 1485680) THEN 5 WHEN (num between 1485681 and 1782816) THEN 6 WHEN (num between 1782817 and 2079952) THEN 7 WHEN (num between 2079953 and 2377088) THEN 8 WHEN (num between 2377089 and 2674224) THEN 9 WHEN (num between 2674225 and 2971364) THEN 10 ELSE NULL END AS decile FROM probab_class_asc_indexed')

In [37]:
#probab_class_asc_index_decili.show(100, False)

In [ ]:
probab_class_asc_index_decili.write.parquet(path_spreading_model_generico)

## Modello Fan

In [38]:
predictions_class_fan = model_fan.transform(final)

In [39]:
from pyspark.ml.feature import VectorSlicer

slicer = VectorSlicer(inputCol="probability", outputCol="prob_1", indices=[1])

predictions_class_fan = slicer.transform(predictions_class_fan)

#predictions_class_generico.select("probability", "prob_1").show()

### Unione 0 con predictions modello fan

In [40]:
#predictions_class_fan.createOrReplaceTempView('fan')

In [41]:
predictions_class_fan1 = predictions_class_fan.drop('features', 'rawPrediction', 'probability', 'prediction',"social_network_class_vec", "tipo_device_class_vec", "ricorrenza_corporate_class_vec", "ricorrenza_tg24_class_vec", "ricorrenza_sport_class_vec",
                      "ricorrenza_appgtv_class_vec", "ricorrenza_appxfactor_class_vec", "ricorrenza_appsport_class_vec", "ricorrenza_appwsc_class_vec",
                      "ricorrenza_corporate_appwsc_appgtv_class_vec", "ricorrenza_editoriale_class_vec", "ricorrenza_web_class_vec", "ricorrenza_app_class_vec",
                      "ricorrenza_totale_class_vec", "social_networkIndex", "tipo_deviceIndex", "ricorrenza_corporateIndex", "ricorrenza_tg24Index", "ricorrenza_sportIndex",
                      "ricorrenza_appgtvIndex", "ricorrenza_appxfactorIndex", "ricorrenza_appsportIndex", "ricorrenza_appwscIndex",
                      "ricorrenza_corporate_appwsc_appgtvIndex", "ricorrenza_editorialeIndex", "ricorrenza_webIndex", "ricorrenza_appIndex",
                      "ricorrenza_totaleIndex")

In [42]:
predictions_class_fan1.createOrReplaceTempView('predictions_class_fan1')
zeri_esclusi.createOrReplaceTempView('zeri_esclusi')

In [43]:
predictions_finale_fan = spark.sql('select * from predictions_class_fan1 union all select * from zeri_esclusi ')

In [44]:
#print(len(zeri_esclusi.columns))

In [45]:
#col_test = predictions_class_fan1.columns
#col_golden_train = zeri_esclusi.columns

#print(len(set(col_test).intersection(col_golden_train)))

In [46]:
predictions_finale_fan.createOrReplaceTempView('predictions_finale_fan')

In [47]:
probab_class_asc_fan = spark.sql('SELECT ext_id, prob_1 FROM predictions_finale_fan ORDER BY prob_1').persist()

In [48]:
#probab_class_asc_fan.count()

In [49]:
probab_class_asc_fan.createOrReplaceTempView('probab_class_asc_fan')
probab_class_asc_indexed_fan = spark.sql('select row_number() over (order by prob_1) as num, * from probab_class_asc_fan').persist()

In [50]:
#probab_class_asc_indexed_fan.show(100)

In [51]:
probab_class_asc_indexed_fan.createOrReplaceTempView('probab_class_asc_indexed_fan')

In [52]:
## ATTENZIONE! Da automatizzare!

probab_class_asc_index_decili_fan = spark.sql('SELECT *, CASE WHEN (num between 0 and 297136) THEN 1 WHEN (num between 297137 and 594272) THEN 2 WHEN (num between 594273 and 891408) THEN 3 WHEN (num between 891409 and 1188544) THEN 4 WHEN (num between 1188545 and 1485680) THEN 5 WHEN (num between 1485681 and 1782816) THEN 6 WHEN (num between 1782817 and 2079952) THEN 7 WHEN (num between 2079953 and 2377088) THEN 8 WHEN (num between 2377089 and 2674224) THEN 9 WHEN (num between 2674225 and 2971364) THEN 10 ELSE NULL END AS decile FROM probab_class_asc_indexed_fan')

In [53]:
#probab_class_asc_index_decili_fan.show(100, False)

In [ ]:
probab_class_asc_index_decili_fan.write.parquet(path_spreading_model_fan)

## Modello Social

In [54]:
predictions_class_social = model_social.transform(final)

In [55]:
from pyspark.ml.feature import VectorSlicer

slicer = VectorSlicer(inputCol="probability", outputCol="prob_1", indices=[1])

predictions_class_social = slicer.transform(predictions_class_social)

#predictions_class_generico.select("probability", "prob_1").show()

### Unione 0 con predictions modello social

In [56]:
#predictions_class_social.createOrReplaceTempView('social')

In [57]:
predictions_class_social1 = predictions_class_social.drop('features', 'rawPrediction', 'probability', 'prediction',"social_network_class_vec", "tipo_device_class_vec", "ricorrenza_corporate_class_vec", "ricorrenza_tg24_class_vec", "ricorrenza_sport_class_vec",
                      "ricorrenza_appgtv_class_vec", "ricorrenza_appxfactor_class_vec", "ricorrenza_appsport_class_vec", "ricorrenza_appwsc_class_vec",
                      "ricorrenza_corporate_appwsc_appgtv_class_vec", "ricorrenza_editoriale_class_vec", "ricorrenza_web_class_vec", "ricorrenza_app_class_vec",
                      "ricorrenza_totale_class_vec", "social_networkIndex", "tipo_deviceIndex", "ricorrenza_corporateIndex", "ricorrenza_tg24Index", "ricorrenza_sportIndex",
                      "ricorrenza_appgtvIndex", "ricorrenza_appxfactorIndex", "ricorrenza_appsportIndex", "ricorrenza_appwscIndex",
                      "ricorrenza_corporate_appwsc_appgtvIndex", "ricorrenza_editorialeIndex", "ricorrenza_webIndex", "ricorrenza_appIndex",
                      "ricorrenza_totaleIndex")

In [58]:
predictions_class_social1.createOrReplaceTempView('predictions_class_social1')
zeri_esclusi.createOrReplaceTempView('zeri_esclusi')

In [59]:
predictions_finale_social = spark.sql('select * from predictions_class_social1 union all select * from zeri_esclusi ')

In [57]:
#print(len(zeri_esclusi.columns))

In [58]:
#col_test = predictions_class_social1.columns
#col_golden_train = zeri_esclusi.columns

#print(len(set(col_test).intersection(col_golden_train)))

In [60]:
predictions_finale_social.createOrReplaceTempView('predictions_finale_social')

In [61]:
probab_class_asc_social = spark.sql('SELECT ext_id, prob_1 FROM predictions_finale_social ORDER BY prob_1').persist()

In [62]:
#probab_class_asc_social.count()

In [63]:
probab_class_asc_social.createOrReplaceTempView('probab_class_asc_social')
probab_class_asc_indexed_social = spark.sql('select row_number() over (order by prob_1) as num, * from probab_class_asc_social').persist()

In [64]:
#probab_class_asc_indexed_social.show(100)

In [65]:
probab_class_asc_indexed_social.createOrReplaceTempView('probab_class_asc_indexed_social')

In [66]:
probab_class_asc_index_decili_social = spark.sql('SELECT *, CASE WHEN (num between 0 and 297136) THEN 1 WHEN (num between 297137 and 594272) THEN 2 WHEN (num between 594273 and 891408) THEN 3 WHEN (num between 891409 and 1188544) THEN 4 WHEN (num between 1188545 and 1485680) THEN 5 WHEN (num between 1485681 and 1782816) THEN 6 WHEN (num between 1782817 and 2079952) THEN 7 WHEN (num between 2079953 and 2377088) THEN 8 WHEN (num between 2377089 and 2674224) THEN 9 WHEN (num between 2674225 and 2971364) THEN 10 ELSE NULL END AS decile FROM probab_class_asc_indexed_social')

In [67]:
#probab_class_asc_index_decili_social.show(100, False)

In [ ]:
probab_class_asc_index_decili_social.write.parquet(path_spreading_model_social)

## UNIONE 3 DATASET CON DECILI



In [67]:
df_generico = spark.read.format('parquet').load("/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE_LUG_DIC/probability_generico_con_decili.parquet").persist()
df_social = spark.read.format('parquet').load("/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE_LUG_DIC/probability_social_con_decili.parquet").persist()
df_fan = spark.read.format('parquet').load("/user/matteo.ballerinipuviani/PREDICTIONSDIGITALIZZAZIONE_LUG_DIC/probability_fan_con_decili.parquet").persist()

In [68]:
df_generico = df_generico.withColumnRenamed('decile', 'decile_generico')
df_fan = df_fan.withColumnRenamed('decile', 'decile_fan')
df_social = df_social.withColumnRenamed('decile', 'decile_social')

In [69]:
df_generico.createOrReplaceTempView('df_generico')
df_fan.createOrReplaceTempView('df_fan')
df_social.createOrReplaceTempView('df_social')

### Unisco in un unico dataset i 3 decili di adobe

In [70]:
df_generico_fan = spark.sql('select df_generico.ext_id, df_generico.decile_generico, df_fan.decile_fan from df_generico left join df_fan on df_generico.ext_id = df_fan.ext_id')

In [71]:
df_generico_fan.show(20)

+--------------------+---------------+----------+
|              ext_id|decile_generico|decile_fan|
+--------------------+---------------+----------+
|000268bb9b4d06e6a...|              4|         1|
|0003fe6fa141cd593...|             10|        10|
|0005a79a8451f3eac...|              2|         3|
|0006ecec67c1a61e3...|              6|         7|
|000b0850c5deb106e...|             10|        10|
|00111532921d68306...|             10|        10|
|0011f1505715ea009...|              7|         7|
|0017cac79130ecafd...|              1|         1|
|001c55fd054d139db...|              2|         2|
|001f9a2827d51dc6e...|              3|         5|
|00248561770c5368d...|              8|         7|
|002c4bdb6f7e352f8...|              9|        10|
|002f6bb23573e0a7e...|              4|         1|
|0032a62cef4ac7a56...|              1|         3|
|003489b29db5b2c06...|              3|         4|
|0049a818c2e0bc46e...|              1|         2|
|004b153b80b8f8109...|              2|         3|


In [72]:
df_generico_fan.createOrReplaceTempView('df_generico_fan')

In [73]:
df_triplo = spark.sql('select df_generico_fan.ext_id, df_generico_fan.decile_generico, df_generico_fan.decile_fan, df_social.decile_social from df_generico_fan left join df_social on df_generico_fan.ext_id = df_social.ext_id')

In [74]:
df_triplo.show()

+--------------------+---------------+----------+-------------+
|              ext_id|decile_generico|decile_fan|decile_social|
+--------------------+---------------+----------+-------------+
|000268bb9b4d06e6a...|              4|         1|            7|
|0003fe6fa141cd593...|             10|        10|            9|
|0005a79a8451f3eac...|              2|         3|            1|
|0006ecec67c1a61e3...|              6|         7|            7|
|000b0850c5deb106e...|             10|        10|            9|
|00111532921d68306...|             10|        10|            9|
|0011f1505715ea009...|              7|         7|            9|
|0017cac79130ecafd...|              1|         1|            2|
|001c55fd054d139db...|              2|         2|            1|
|001f9a2827d51dc6e...|              3|         5|            1|
|00248561770c5368d...|              8|         7|            5|
|002c4bdb6f7e352f8...|              9|        10|            8|
|002f6bb23573e0a7e...|              4|  

### Creo il decile pesato di Generico, Fan, Social:

In [75]:
df_triplo.createOrReplaceTempView('df_triplo')

In [76]:
df_4 = spark.sql('select ext_id, decile_generico, decile_fan, decile_social, sum(decile_generico*0.5 + decile_fan*0.2 + decile_social*0.3) as score from df_triplo group by ext_id, decile_generico, decile_fan, decile_social')

In [ ]:
df_4.write.parquet("/user/matteo.ballerinipuviani/df_4_decili_adobe.parquet")

In [97]:
#df_4.show(100, False )

+------------------------------------------------+---------------+----------+-------------+-----+
|ext_id                                          |decile_generico|decile_fan|decile_social|score|
+------------------------------------------------+---------------+----------+-------------+-----+
|000268bb9b4d06e6ac81329a96fcc744e05c79835496da4b|4              |1         |7            |4.3  |
|0003fe6fa141cd593acb65361edd4832c6ca28d9a70e9aba|10             |10        |9            |9.7  |
|0005a79a8451f3eac98207d3d3111926e4e5a15a13e7d093|2              |3         |1            |1.9  |
|0006ecec67c1a61e374022c955e98b3ffa326783316cf9aa|6              |7         |7            |6.5  |
|000b0850c5deb106e8528933477f8de336ecc62ec46eb24c|10             |10        |9            |9.7  |
|00111532921d6830675cab7cc4d3b70dbee2b5a6e78fc08d|10             |10        |9            |9.7  |
|0011f1505715ea0090fcdda1fa4ca380c8a3c9c00aeaaa93|7              |7         |9            |7.6  |
|0017cac79130ecafd14

In [113]:
#df_4.count()

In [77]:
df_4.createOrReplaceTempView('df_4')

In [115]:
#df_4_ordered = spark.sql('select score, count(ext_id) from df_4 group by score order by score asc')

In [ ]:
#df_4_ordered.show(40)

In [114]:
#df_4.printSchema()

In [78]:
df_score = spark.sql('select ext_id, decile_generico, decile_fan, decile_social, score from df_4 order by score asc')

In [106]:
#df_score.show(100)

+--------------------+---------------+----------+-------------+-----+
|              ext_id|decile_generico|decile_fan|decile_social|score|
+--------------------+---------------+----------+-------------+-----+
|50f2aa3d72c194bbf...|              1|         2|            1|  1.2|
|8818adbf0f0de243f...|              1|         2|            1|  1.2|
|50ff059fa34fbc5db...|              1|         2|            1|  1.2|
|00924fa0f071e8d74...|              1|         2|            1|  1.2|
|52d09b3afde4849d2...|              1|         2|            1|  1.2|
|01751ab631e923636...|              1|         2|            1|  1.2|
|52fa9db906edbd7a8...|              1|         2|            1|  1.2|
|03de3d4f993db45e7...|              1|         2|            1|  1.2|
|53be88bbf59924986...|              1|         2|            1|  1.2|
|052d9256280aaa211...|              1|         2|            1|  1.2|
|53cd5eaaf70d84fc6...|              1|         2|            1|  1.2|
|057429a7a03ac29fc..

In [79]:
df_score = df_score.withColumn("score", df_score.score.cast("double"))

In [80]:
from pyspark.ml.feature import QuantileDiscretizer

In [81]:
discretizer = QuantileDiscretizer(numBuckets=10, inputCol="score", outputCol="bin_score")
df_score_bin = discretizer.fit(df_score).transform(df_score)

In [109]:
#df_score_bin.show(100)

+--------------------+---------------+----------+-------------+-----+---------+
|              ext_id|decile_generico|decile_fan|decile_social|score|bin_score|
+--------------------+---------------+----------+-------------+-----+---------+
|4d0888f6bd38207ba...|              1|         2|            1|  1.2|      0.0|
|6208df70d40493292...|              1|         2|            1|  1.2|      0.0|
|6ac851f976aefb641...|              1|         2|            1|  1.2|      0.0|
|57711e691e9dc948f...|              1|         2|            1|  1.2|      0.0|
|2d57d161a5bc795f2...|              1|         2|            1|  1.2|      0.0|
|683d0ad087638de79...|              1|         2|            1|  1.2|      0.0|
|4bb3d22ccd78f6a70...|              1|         2|            1|  1.2|      0.0|
|545627a61e8c53c99...|              1|         2|            1|  1.2|      0.0|
|13156a38279872c7b...|              1|         2|            1|  1.2|      0.0|
|006616f2580451bf0...|              1|  

In [82]:
df_score_bin.createOrReplaceTempView('df_score_bin')

In [111]:
#df_score_bin_grouped = spark.sql('select (bin_score+1) as bin_score, count(ext_id) from df_score_bin group by bin_score order by bin_score asc')

In [112]:
#df_score_bin_grouped.show(15)

+---------+-------------+
|bin_score|count(ext_id)|
+---------+-------------+
|      1.0|       278012|
|      2.0|       284553|
|      3.0|       285745|
|      4.0|       291653|
|      5.0|       266237|
|      6.0|       305911|
|      7.0|       275558|
|      8.0|       294932|
|      9.0|       262109|
|     10.0|       316959|
+---------+-------------+



## Unisco tutte le variabili di Adobe con decili Generico, Fan, Social e score

In [83]:
df_originale = spark.read.format('parquet').load(path_adobe).persist()

In [84]:
df_originale.createOrReplaceTempView('df_originale')
df_score_bin.createOrReplaceTempView('df_score_bin')

In [85]:
df_joined = spark.sql('select df_originale.*,df_score_bin.decile_generico, df_score_bin.decile_fan, df_score_bin.decile_social, df_score_bin.score, (df_score_bin.bin_score + 1) as bin_score from df_originale left join df_score_bin on df_originale.ext_id = df_score_bin.ext_id')

In [86]:
df_joined = df_joined.drop('score_digital')

In [136]:
#df_joined.printSchema()

In [ ]:
df_joined.write.parquet("/user/matteo.ballerinipuviani/df_score_digitale_adobe.parquet", mode = "overwrite")

## Unisco i decili di Adobe ed Adform 

In [87]:
df_adobe = spark.read.format('parquet').load("/user/matteo.ballerinipuviani/df_score_digitale_adobe.parquet").persist()

In [88]:
df_adform = spark.read.format('parquet').load("/user/emma.cambieri/PROBABILITY_DECILI_DIGITALIZZAZIONE/probability_adform_con_decili.parquet").persist()

In [139]:
#df_adform.printSchema()

In [143]:
#df_adobe.printSchema()

In [89]:
df_adobe.createOrReplaceTempView('df_adobe')
df_adform.createOrReplaceTempView('df_adform')

In [145]:
df_finale_digitalizzazione = spark.sql('select coalesce(df_adform.ext_id_impression, df_adobe.ext_id) as ext_id_adobe_adform, df_adform.ext_id_impression as ext_id_adform, df_adobe.ext_id as ext_id_adobe, df_adform.decile as adform_decile, df_adobe.score as score_adobe, df_adobe.bin_score as adobe_decile, df_adobe.decile_generico, df_adobe.decile_fan, df_adobe.decile_social from df_adform full outer join df_adobe on df_adform.ext_id_impression = df_adobe.ext_id')

AnalysisException: "cannot resolve '`df_adobe.decile_generico`' given input columns: [visite_weekend_sport, visite_appsport_notte, ricorrenza_appxfactor, ricorrenza_appgtv, cookie_appwsc, secondi_editoriale, media_visite_sport, media_visite_appxfactor, visite_corporate_sera, ricorrenza_totale, visite_weekend_appgtv, visite_corporate_notte, visite_editoriale, ricorrenza_appsport, numero_reportsuite, numero_mesi_appwsc, durata_media_visite_tg24, visite_tg24_sera, media_visite_appgtv, score_utilizzo, visite_sport_notte, max_mesi_web, percentuale_mobile, visite_settimana_sport, ricorrenza_web, ricorrenza_tg24, visite_sport_pomeriggio, count_visits_other, decile, visite_weekend_appwsc, score, visite_appgtv_notte, numero_mesi_sport, visite_settimana_appwsc, secondi_totali_tg24, avg_visite_mese_web, visite_totali_sport, visite_totali_appgtv, visite_totali_appwsc, visite_appgtv_sera, visite_settimana_corporate, visite_appwsc_sera, visite_appsport_mattina, durata_media_visite_corporate, visite_

In [ ]:
df_finale_digitalizzazione.write.parquet("/user/matteo.ballerinipuviani/DF_CON_SCORE/df_adform_adobe_lug_dic.parquet", mode = "overwrite")

In [163]:
df_adobe_adform = spark.read.format('parquet').load("/user/matteo.ballerinipuviani/DF_CON_SCORE/df_adform_adobe_lug_dic.parquet").persist()

In [90]:
#df_adobe_adform.printSchema()

In [165]:
df_adobe_adform.show()

+--------------------+--------------------+--------------------+-------------+-----------+------------+
| ext_id_adobe_adform|       ext_id_adform|        ext_id_adobe|adform_decile|score_adobe|adobe_decile|
+--------------------+--------------------+--------------------+-------------+-----------+------------+
|0005f3f18bda8a53a...|                null|0005f3f18bda8a53a...|         null|        5.2|         5.0|
|0006530d860a31ca2...|0006530d860a31ca2...|0006530d860a31ca2...|            8|        8.7|         9.0|
|0006b83f9d38ac301...|0006b83f9d38ac301...|0006b83f9d38ac301...|            3|        1.9|         2.0|
|000ae1a773010077d...|000ae1a773010077d...|                null|            1|       null|        null|
|000aec8698c204eca...|000aec8698c204eca...|000aec8698c204eca...|            7|        9.1|        10.0|
|000effd99733aebfe...|000effd99733aebfe...|000effd99733aebfe...|            3|        3.8|         3.0|
|000fd3e0c929462b3...|000fd3e0c929462b3...|000fd3e0c929462b3...|

In [166]:
df_adobe_adform.createOrReplaceTempView('df_adobe_adform')

In [170]:
decile_pesato_50_50 = spark.sql('SELECT D.*,CASE WHEN (D.adobe_decile is null) THEN D.adform_decile WHEN (D.adform_decile is null) THEN D.adobe_decile WHEN (D.adobe_decile is not null and D.adform_decile is not null) THEN (round(0.5*D.adobe_decile + 0.5*D.adform_decile)) ELSE Null END AS decile_pesato FROM df_adobe_adform D')

In [171]:
decile_pesato_50_50.show()

+--------------------+--------------------+--------------------+-------------+-----------+------------+-------------+
| ext_id_adobe_adform|       ext_id_adform|        ext_id_adobe|adform_decile|score_adobe|adobe_decile|decile_pesato|
+--------------------+--------------------+--------------------+-------------+-----------+------------+-------------+
|0005f3f18bda8a53a...|                null|0005f3f18bda8a53a...|         null|        5.2|         5.0|          5.0|
|0006530d860a31ca2...|0006530d860a31ca2...|0006530d860a31ca2...|            8|        8.7|         9.0|          9.0|
|0006b83f9d38ac301...|0006b83f9d38ac301...|0006b83f9d38ac301...|            3|        1.9|         2.0|          3.0|
|000ae1a773010077d...|000ae1a773010077d...|                null|            1|       null|        null|          1.0|
|000aec8698c204eca...|000aec8698c204eca...|000aec8698c204eca...|            7|        9.1|        10.0|          9.0|
|000effd99733aebfe...|000effd99733aebfe...|000effd99733a

In [172]:
#decile_pesato_50_50.createOrReplaceTempView('decile_pesato_50_50')

In [173]:
#count_decili = spark.sql('select decile_pesato, count(ext_id_adobe_adform) from decile_pesato_50_50 group by decile_pesato order by decile_pesato asc')

In [174]:
#count_decili.show()

+-------------+--------------------------+
|decile_pesato|count(ext_id_adobe_adform)|
+-------------+--------------------------+
|          1.0|                    203032|
|          2.0|                    329763|
|          3.0|                    326856|
|          4.0|                    383638|
|          5.0|                    411647|
|          6.0|                    423857|
|          7.0|                    405673|
|          8.0|                    348468|
|          9.0|                    276864|
|         10.0|                    211977|
+-------------+--------------------------+



In [175]:
#decile_pesato_50_50.count()

3321775

In [ ]:
decile_pesato_50_50.write.parquet("/user/matteo.ballerinipuviani/DF_CON_SCORE/SCORE_DIGITALE_PESATO_ADFORM_ADOBE_50_50")

In [179]:
df_joined = spark.read.format('parquet').load("/user/matteo.ballerinipuviani/DF_CON_SCORE/SCORE_DIGITALE_PESATO_ADFORM_ADOBE_50_50").persist()

In [180]:
#df_joined.printSchema()

root
 |-- ext_id_totale: string (nullable = true)
 |-- numero_mesi_corporate: double (nullable = true)
 |-- numero_mesi_tg24: double (nullable = true)
 |-- numero_mesi_sport: double (nullable = true)
 |-- numero_mesi_appgtv: double (nullable = true)
 |-- numero_mesi_appxfactor: double (nullable = true)
 |-- numero_mesi_appsport: double (nullable = true)
 |-- numero_mesi_appwsc: double (nullable = true)
 |-- visite_totali_corporate: double (nullable = true)
 |-- visite_totali_sport: double (nullable = true)
 |-- visite_totali_tg24: double (nullable = true)
 |-- visite_totali_appwsc: double (nullable = true)
 |-- visite_totali_appsport: double (nullable = true)
 |-- visite_totali_appgtv: double (nullable = true)
 |-- visite_totali_appxfactor: double (nullable = true)
 |-- numero_reportsuite: double (nullable = true)
 |-- cookie_corporate: double (nullable = true)
 |-- cookie_tg24: double (nullable = true)
 |-- cookie_sport: double (nullable = true)
 |-- cookie_skyitdev: double (nullable 